We have a lot of ways to ensure our code does not contain our authentication credentials and the two most common ways of doing this are through environment files or using a "credential vault" of sorts. 

## Using an Environment File

*Create a `.env` file. 
The .env file should be setup with variable values like so:

VARIABLE_NAME = value </br>
KEY_NAME = value

In [ ]:
#Import packages
from dotenv import load_dotenv
import os

load_dotenv() #if your env file changes after running this, you need to reset kernal for changes to propogate

#Get ENV file variables
api_key = os.getenv('KEY') #os.environ.get('KEY')
base_url = os.getenv('ENDPOINT') #os.environ.get('ENDPOINT')

False

# Start with LangDetect SDK code we had from before
-- maybe copy it into a new dir

%pip install azure-ai-textanalytics==5.3.0 <br>
%pip install azure-identity==1.5.0 <br>
%pip install azure-keyvault-secrets==4.2.0

### Key Vault

Even with the .env files, we are putting our keys into our code. Or into our directories <br>
We can avoid putting the keys here, by using Azure Key Vault. It adds an extra layer of authentication. <br>
We have to authenticate to the key vault. Then get the key & connect to the service -- adding an extra layer of security

In Key vault, we store our keys, secrets, and certificates. <br><br>
A secret is anything that you want to tightly control access to, such as API keys, passwords, sometimes even other certificates, or cryptographic keys. <br>
The Azure AI service resource keys are actually considered secrets and are very easy to store in the vault <br><br>
Now, there are a few ways we can secure the key vault itself. For example, <br>
we can use role based access contol to give permissions like key administrator, 

Here are other important terms: <br>

- **Tenant**: A tenant is the organization that owns and manages a specific instance of Microsoft cloud services. It's most often used to refer to the set of Azure and Microsoft 365 services for an organization.
- **Security principal**: An Azure security principal is a security identity that user-created apps, services, and automation tools use to access specific Azure resources. Think of it as a "user identity" (username and password or certificate) with a specific role, and tightly controlled permissions. A security principal should only need to do specific things, unlike a general user identity. It improves security if you grant it only the minimum permission level that it needs to perform its management tasks. A security principal used with an application or service is called a service principal. <br> In essence, by using a Service Principal, you avoid creating “fake users” (we would call them service account in on-premises Active Directory…) in Azure AD to manage authentication when you need to access Azure Resources.
- **Managed identities**: Azure Key Vault provides a way to securely store credentials and other keys and secrets, but your code needs to authenticate to Key Vault to retrieve them. Using a managed identity makes solving this problem simpler by giving Azure services an automatically managed identity in Microsoft Entra ID. <br>
Managed Identities are in essence 100% identical in functionality and use case than Service Principals. In fact, they are actually Service Principals.

What makes them different though, is: – They are always linked to an Azure Resource, not to an application or 3rd party connector – They are automatically created for you, including the credentials; big benefit here is that no one knows the credentials

Managed Identities exist in 2 formats: – System assigned; in this scenario, the identity is linked to a single Azure Resource, eg a Virtual Machine, a Logic App, a Storage Account, Web App, Function,… so almost anything. Next, they also “live” with the Azure Resource, which means they get deleted when the Azure Resource gets deleted. – User Assigned Managed Identity, which means that you first have to create it as a stand-alone Azure resource by itself, after which it can be linked to multiple Azure Resources. 

### Networks to protect Endpoints

As of right now, our endpoints are publicly available, meaning anyone with access to them can use them. <br>
We have our keys protected, but we should probably protect our endpoints too, ya? <br>
What we can do is restrict this endpoint via a virtual network. Now these endpoints are only available to devices which have access to that network.

### Containers

Azure AI services need internet access to function <br>
but what about remote locations that have intermittent or limited internet connectivity. <br>
OR what if there are governance rules preventing you from uploading data to a cloud provider <br><br>
-- *enter containers* --

When you deploy a software service, it must be hosted in an environment that provides the hardware: the OS and supporting runtime components. <br>
Azure AI services are provided as cloud services in which the service software is hosted in an Azure datacenter <br>
We could also deploy an Azure AI service in a container which encapsulates the necessary runtime components. <br>
This is then deployed in a container host that provides the OS and hardware.. These container instances are called **images** <br>
Containers are not all encompassing. for example, there is not one container for all of the Language services



To use a container, you typically pull a container image from a registry, provide your specific configuration settings, and deploy it to a container host.

### Key Vault Demo

- have an Azure AI service (single or multi)
- Create an azure key vault
    - ensure that in the access configuration tab, you have 'vault access policy' selected -- not RBAC
    - check your name in the access policies


- Add the api key to the key vault as a secret
    - In the key vault resource, expand objects
    - create a new secret
    - save the AI service key in the key vault
    - *Note: you may need to change the networking settings of the key vault to allow access from all networks

Discussion: How is this more secure than just creating a .env file?
   - Now, we have to provide an identity to Azure to get authenticated to use the key vault and get that key value 

#### We need a way for our vscode code to access the keys in azure key vault, so we need a service principal

- Create a Service Principal (Application Registration)
    - *Navigate to Entra and find the Enterprise Applications section
        - Maybe go to the docs for an "application registration"
    - Name: AI-Language-AppRegistration
    - Supported account types: Choose "Accounts in this organizational directory only".
    - click register

*Again, all we really did here was create an identity inside of Azure


- In the manage section of the left navbar, go to certificates and secrets
    - Generate a New Client Secret
    - Copy the value of the client secret. 




#### With Key Vault (lik all Azure resources) we need to dictate who has access to it
- Assign key vault access policies to the service principal
    - go to key vault
    - In the Key Vault's left-hand menu, click on "Access policies".
    - Click on "Create".
    - 1 - Permissions: Get and List
    - 2 - Principal: (use the name or paste the objectID to find the service principal)
    - 3 - Application: *none*
    - 4 - create
    OR (if using RBAC)
    - go to key vault
    - click on IAM
    - key vault reader
    - give access to the Service Principal

- We are going to alter our language demo script
- note the environment variables we will now use (put these in your .env file)
    - **COG_SERVICE_ENDPOINT**: The endpoint for your Azure AI Services resource
    - **KEY_VAULT**: The name of your Azure Key Vault resource
    - **TENANT_ID**: The tenant for your service principal
    - **APP_ID**: The appId for your service principal
    - **APP_PASSWORD**: The password for your service principal (created in certificates of the app registration)




Let's try it out!!

In [ ]:
#may need to 
!pip install azure-identity
!pip install azure-keyvault-secrets

In [ ]:
# Import packages
# - for env file
from dotenv import load_dotenv
import os

# - from lang demo (SDK)
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# -to get needed info from azure
from azure.keyvault.secrets import SecretClient
from azure.identity import ClientSecretCredential


# Depending on how you want to grant access, yo ucould tweak this exercise to use the user's Azure credentials
# from azure.identity import DefaultAzureCredential


In [2]:
load_dotenv()

True

In [ ]:
language_endpoint = os.environ.get('COG_SERVICE_ENDPOINT')

key_vault_name = os.getenv('KEY_VAULT')
app_tenant = os.getenv('TENANT_ID')
app_id = os.getenv('APP_ID') #client id
app_password = os.getenv('APP_PASSWORD')

# Get Azure AI services key from keyvault using the service principal credentials
key_vault_uri = f"https://{key_vault_name}.vault.azure.net/"

# refer to the Language SDK script
""" 
def authenticate_client():
    ta_credential = AzureKeyCredential(ai_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=ai_endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()
 """
#to do this, we need to connect to our service principal, then connect to the key vault, and get the secret from the key vault


credential = ClientSecretCredential(tenant_id=app_tenant, client_id=app_id, client_secret=app_password) #access to the Service Principal "authenticates the Service Principal"
keyvault_client = SecretClient(key_vault_uri, credential) #allows us to connect to the key vault
secret_key = keyvault_client.get_secret("Cognitive-Services-Key") #now we need to get the secret from the key vault -- this is the name of the secret in key vault
cog_key = secret_key.value #the above command gets the "entire" secret -- more than just the value


#now, we do the same auth as before
credential = AzureKeyCredential(cog_key)
client = TextAnalyticsClient(endpoint=language_endpoint, credential=credential)

In [4]:
# Example method for detecting the language of text
def language_detection_example():
    try:
        userText = ''
        while userText.lower() != 'quit':
            userText = input('\nEnter some text ("quit" to stop)\n')
            if userText.lower() != 'quit':
                documents = [userText]
                response = client.detect_language(documents = documents, country_hint = 'us')[0]
                print("Language: ", response.primary_language.name)

    except Exception as err:
        print("Encountered exception. {}".format(err))

In [5]:
language_detection_example()

Language:  English


### Further discussion: Protecting the endpoint

- Using networking

### Container demo

- create a new container instance
    - image source: other registry
    - Image: *get link from docker hub (Text Analytics Language)
    - change size of the image: 4 cores and 8 gigs memory (no gpu)
    Networking
    - (optional) give a DNS name label
    Advanced
    - Restart policy: always
    - Environment Variables:
        - Secure: yes ; ApiKey: ~~~
        - Secure: no ; Billing: ~endpoint url~
        - Secure: no ; Eula: Accept
-  make sure container is running
-  if it is running, you can use it like any web application
    -  *you can use either the IP address or use the Fully Qualified Domain Name (FQDN)
    - Paste the FQDN into your browser and append ":5000"
    - now, you can make rest requests from this container
    - *you could either create your own Frontend to do this or just
    - append "/swagger" ("/swagger/index.html")
    - *We can use this envronment to test the service

    - down at the ready section you can see if the service is up and running